# Combining everything
  > pls use the r envirnment


## import and setup


In [ ]:
import os;
pwd = os.getcwd;
cd  = os.chdir;
ls  = os.listdir;

import re;

import pandas as pd;
import numpy as np;
from matplotlib import pyplot as plt;
from matplotlib import dates as mdates;
import seaborn as sns;

import rpy2;
%load_ext rpy2.ipython

import rpy2.robjects as ro;

from rpy2.robjects import pandas2ri;
from rpy2.robjects import default_converter
from rpy2.robjects.conversion import localconverter;

from rpy2.robjects.packages import importr;

utils = importr("utils");
grdevices = importr('grDevices');
infenergy = importr('infenergy');


def help_r( python_name ):
    print( str( utils.help( python_name.replace('_','.') ) ) );




## Read all from `forum_electricity` database
i.e. time: "2013-01-21 17:40:01 GMT" "2020-06-18 07:20:05 GMT" ( pls update sm other time )

and print basic Infos.


In [ ]:
data = infenergy.get_inf_meter_data("2013-01-21", "2020-06-19"); # as an r dataframe

with localconverter(ro.default_converter + pandas2ri.converter):
  data_converted   = ro.conversion.rpy2py( data );


data_converted = data_converted.reset_index( drop=True );
data_converted.time = data_converted.time.dt.tz_convert('GMT');

data_converted.info()


ax = data_converted.set_index('time')['2014-1-30':'2014-1-30'].diff().rename(columns={'cumkwh':'diff'}).dropna().plot()
ax.set_xlabel('time');
ax.set_ylabel('diff(cumkwh)');
plt.show();



In [ ]:

data_ups_AB_hourly = infenergy.get_ups_hourly("2013-01-21", "2020-06-19", ['forumA','forumB'])

with localconverter(ro.default_converter + pandas2ri.converter):
  data_ups_AB_converted   = ro.conversion.rpy2py( data_ups_AB_hourly );


data_ups_AB_converted = data_ups_AB_converted.reset_index( drop=True );
data_ups_AB_converted.Time = data_ups_AB_converted.Time.dt.tz_convert('GMT');

#data_ups_converted.dropna().shape
#(data_ups_converted.dropna() == data_ups_AB_converted.dropna()).sum()



ax = data_ups_AB_converted.set_index('Time').rename(columns={'cumkwh':'diff'}).resample('W').sum().dropna().plot()
ax.set_xlabel('Time');
ax.set_ylabel('diff(cumkwh)');
plt.show();




## Read all from `../data/preprocessed/` and join with the data above



In [ ]:

data_forum_electricity = data_converted;

data_forum_electricity.head()
data_forum_electricity.tail()

# datetime type column as index helps joining and concating data

gogar_2013 = pd.read_csv('~/Desktop/work/DS MSc/THESIS/code/data/preprocessed/WH/gogar_2013.csv', index_col='OB_TIME')
gogar_2014 = pd.read_csv('~/Desktop/work/DS MSc/THESIS/code/data/preprocessed/WH/gogar_2014.csv', index_col='OB_TIME')
gogar_2015 = pd.read_csv('~/Desktop/work/DS MSc/THESIS/code/data/preprocessed/WH/gogar_2015.csv', index_col='OB_TIME')
gogar_2016 = pd.read_csv('~/Desktop/work/DS MSc/THESIS/code/data/preprocessed/WH/gogar_2016.csv', index_col='OB_TIME')
gogar_2017 = pd.read_csv('~/Desktop/work/DS MSc/THESIS/code/data/preprocessed/WH/gogar_2017.csv', index_col='OB_TIME')
gogar_2018 = pd.read_csv('~/Desktop/work/DS MSc/THESIS/code/data/preprocessed/WH/gogar_2018.csv', index_col='OB_TIME')
gogar_2019 = pd.read_csv('~/Desktop/work/DS MSc/THESIS/code/data/preprocessed/WH/gogar_2019.csv', index_col='OB_TIME')
gogar_2020 = pd.read_csv('~/Desktop/work/DS MSc/THESIS/code/data/preprocessed/WH/gogar_2020.csv', index_col='OB_TIME')

gogar_rain_2013 = pd.read_csv('~/Desktop/work/DS MSc/THESIS/code/data/preprocessed/RH/gogar_rain_2013.csv', index_col='OB_END_TIME')
gogar_rain_2014 = pd.read_csv('~/Desktop/work/DS MSc/THESIS/code/data/preprocessed/RH/gogar_rain_2014.csv', index_col='OB_END_TIME')
gogar_rain_2015 = pd.read_csv('~/Desktop/work/DS MSc/THESIS/code/data/preprocessed/RH/gogar_rain_2015.csv', index_col='OB_END_TIME')
gogar_rain_2016 = pd.read_csv('~/Desktop/work/DS MSc/THESIS/code/data/preprocessed/RH/gogar_rain_2016.csv', index_col='OB_END_TIME')
gogar_rain_2017 = pd.read_csv('~/Desktop/work/DS MSc/THESIS/code/data/preprocessed/RH/gogar_rain_2017.csv', index_col='OB_END_TIME')
gogar_rain_2018 = pd.read_csv('~/Desktop/work/DS MSc/THESIS/code/data/preprocessed/RH/gogar_rain_2018.csv', index_col='OB_END_TIME')
gogar_rain_2019 = pd.read_csv('~/Desktop/work/DS MSc/THESIS/code/data/preprocessed/RH/gogar_rain_2019.csv', index_col='OB_END_TIME')
gogar_rain_2020 = pd.read_csv('~/Desktop/work/DS MSc/THESIS/code/data/preprocessed/RH/gogar_rain_2020.csv', index_col='OB_END_TIME')

 
gogar_rain = pd.concat([gogar_rain_2013, gogar_rain_2014, gogar_rain_2015, gogar_rain_2016, gogar_rain_2017, gogar_rain_2018, gogar_rain_2019, gogar_rain_2020])
gogar_WH   = pd.concat([gogar_2013, gogar_2014, gogar_2015, gogar_2016, gogar_2017, gogar_2018, gogar_2019, gogar_2020])

gogar_rain.index = pd.to_datetime(gogar_rain.index).tz_localize(tz='GMT')
gogar_WH.index = pd.to_datetime(gogar_WH.index).tz_localize(tz='GMT')


gogar_WH.shape
nonEmpties_count = (gogar_WH == ' ').sum( axis = 0 )
useful_columns = [ column for column in gogar_WH.columns if ( 
    column not in pd.DataFrame( nonEmpties_count[ nonEmpties_count > 64946/2 ] ).T.columns and 
        ( re.search("ID", column) == None and 
            re.search("_Q", column) == None and
            re.search("_J", column) == None
        ) and 
    column not in ['MET_DOMAIN_NAME', 'VERSION_NUM', 'REC_ST_IND', 'SRC_OPR_TYPE','METO_STMP_TIME']
    )]


gogar_rain.shape
rain_nonEmpties_count = (gogar_rain == ' ').sum( axis = 0 )
rain_useful_columns = [ column for column in gogar_rain.columns if ( 
    column not in pd.DataFrame( rain_nonEmpties_count[ rain_nonEmpties_count > 64992/2 ] ).T.columns and 
        ( re.search("ID", column) == None and 
            re.search("_Q", column) == None and
            re.search("_J", column) == None
        ) and 
    column not in ['MET_DOMAIN_NAME', 'VERSION_NUM', 'REC_ST_IND', 'METO_STMP_TIME', 'OB_HOUR_COUNT']
    )]
#gogar_WH[['AIR_TEMPERATURE','RLTV_HUM','DEWPOINT','WETB_TEMP','WMO_HR_SUN_DUR','MSL_PRESSURE','STN_PRES','WIND_SPEED','WIND_DIRECTION','VISIBILITY','CLD_BASE_HT']]

#updated 5th August
gogar_weather = gogar_WH[useful_columns].join(gogar_rain[rain_useful_columns])

gogar_weather['2018']

gogar_rain[rain_useful_columns]['PRCP_AMT'].value_counts()


all useful attributes found so far are concatinated and joined to the `gogar_weather` variable

Since `forum_electricity` is sampled in roughly each 10 minutes, we need to re bin it to join to the gogar data.

# Re-binning `forum_electricity`



In [ ]:

# As we can see, interval is too irregular and to fine-grined for the GOGAR data.
print( data_forum_electricity )



# proposal
data_forum_electricity_resampled = data_forum_electricity.set_index('time').resample('H').max();

data_forum_electricity_resampled.shape

data_forum_electricity_resampled.interpolate(method='linear').isna().cumkwh.value_counts()

data_forum_electricity_resampled_interpolated = data_forum_electricity_resampled.interpolate(method='linear');

data_forum_electricity_resampled_interpolated_diff = data_forum_electricity_resampled_interpolated.diff().dropna().rename(columns={'cumkwh':'bin_kwh'})

# confirm quality
fig, axs = plt.subplots(5, 2, figsize=(15,15))


data_forum_electricity_resampled_interpolated_diff['2014-4-30':'2015-4-30'].plot(ax=axs[0, 0])
data_forum_electricity.set_index('time')['2014-4-30':'2015-4-30'].diff(periods=6).rename(columns={'cumkwh':'diff'}).dropna().plot(ax=axs[0, 1])
axs[0, 0].set_ylabel('1 year')

data_forum_electricity_resampled_interpolated_diff['2014-4-30':'2014-11-20'].plot(ax=axs[1, 0])
data_forum_electricity.set_index('time')['2014-4-30':'2014-11-20'].diff(periods=6).rename(columns={'cumkwh':'diff'}).dropna().plot(ax=axs[1, 1])
axs[1, 0].set_ylabel('6 months')

data_forum_electricity_resampled_interpolated_diff['2014-4-30':'2014-7-29'].plot(ax=axs[2, 0])
data_forum_electricity.set_index('time')['2014-4-30':'2014-7-29'].diff(periods=6).rename(columns={'cumkwh':'diff'}).dropna().plot(ax=axs[2, 1])
axs[2, 0].set_ylabel('3 months quarter')

data_forum_electricity_resampled_interpolated_diff['2014-4-30':'2014-5-13'].plot(ax=axs[3, 0])
data_forum_electricity.set_index('time')['2014-4-30':'2014-5-13'].diff(periods=6).rename(columns={'cumkwh':'diff'}).dropna().plot(ax=axs[3, 1])
axs[3, 0].set_ylabel('2 weeks')

data_forum_electricity_resampled_interpolated_diff['2014-4-30':'2014-4-30'].plot(ax=axs[4, 0])
data_forum_electricity.set_index('time')['2014-4-30':'2014-4-30'].diff(periods=6).rename(columns={'cumkwh':'diff'}).dropna().plot(ax=axs[4, 1])
axs[4, 0].set_ylabel('1 day')

fig.tight_layout()
plt.show()



data_forum_electricity_resampled_interpolated_diff.describe()

# NEW VERSION: 2020 august 12, already fixed tz problem, COMMENTING OUT
# fix tz problem
# gogar_weather.index = pd.to_datetime(gogar_weather.index).tz_localize(tz='GMT')
gogar_weather.info()

gogar_weather.index[0] #Timestamp('2013-01-01 00:00:00+0000', tz='GMT')
gogar_weather.index[-1] #Timestamp('2020-05-31 23:00:00+0000', tz='GMT')
combined_data_with_nans = gogar_weather.join( data_forum_electricity_resampled_interpolated_diff )['2013-01-21 18':'2020-6-1']
combined_data_with_nans = combined_data_with_nans.replace(' ',np.nan)
for feature in combined_data_with_nans:
    print(feature)
    combined_data_with_nans[feature] = pd.to_numeric(combined_data_with_nans[feature])

# Combined data: `combined_data_with_nans`

## EDA:


In [ ]:

combined_data_with_nans.info()

combined_data_with_nans.describe()

combined_data_with_nans.shape

fig, axs = plt.subplots(4, 4, figsize=(15,15))
combined_data_with_nans['AIR_TEMPERATURE'].plot(ax=axs[0,0], title='AIR_TEMPERATURE')
combined_data_with_nans['RLTV_HUM'].plot(ax=axs[0,1], title='RLTV_HUM')
combined_data_with_nans['DEWPOINT'].plot(ax=axs[0,2], title='DEWPOINT')
combined_data_with_nans['WETB_TEMP'].plot(ax=axs[0,3], title='WETB_TEMP')
combined_data_with_nans['WMO_HR_SUN_DUR'].plot(ax=axs[1,0], title='WMO_HR_SUN_DUR')
combined_data_with_nans['MSL_PRESSURE'].plot(ax=axs[1,1], title='MSL_PRESSURE')
combined_data_with_nans['STN_PRES'].plot(ax=axs[1,2], title='STN_PRES')
combined_data_with_nans['WIND_SPEED'].plot(ax=axs[1,3], title='WIND_SPEED')
combined_data_with_nans['WIND_DIRECTION'].plot(ax=axs[2,0], title='WIND_DIRECTION')
combined_data_with_nans['VISIBILITY'].plot(ax=axs[2,1], title='VISIBILITY')
combined_data_with_nans['CLD_BASE_HT'].plot(ax=axs[2,2], title='CLD_BASE_HT')
combined_data_with_nans['PRCP_AMT'].plot(ax=axs[2,3], title='PRCP_AMT')
combined_data_with_nans['bin_kwh'].plot(ax=axs[3,0], title='bin_kwh')

fig.tight_layout()

plt.show()

plt.figure(figsize=(10,10))
sns.heatmap(
    combined_data_with_nans.dropna().corr(),
    cmap=plt.cm.coolwarm,
    vmax=1.0,
    linewidths=0.1,
    linecolor='white',
    square=True,
    annot=True
)



# Write to files



In [ ]:

# commented out because I CAN haha ( fr io can be slow and prone to errors )

# data_forum_electricity.to_csv( path_or_buf='../data/preprocessed/UOE/data_forum_electricity.csv')
# data_forum_electricity_resampled_interpolated.to_csv( path_or_buf='../data/preprocessed/UOE/data_forum_electricity_resampled_interpolated.csv')
# data_forum_electricity_resampled_interpolated_diff.to_csv( path_or_buf='../data/preprocessed/UOE/data_forum_electricity_resampled_interpolated_diff.csv')
# combined_data_with_nans.to_csv( path_or_buf='/Users/lautinyeung/Desktop/work/DS MSc/THESIS/code/data/preprocessed/combined/combined_data_with_nans.csv')


# L R test


In [ ]:
from sklearn.linear_model import LinearRegression;
from sklearn.model_selection import train_test_split;



combined_data_dropped = combined_data_with_nans.dropna()

combined_data_dropped.shape
features = combined_data_dropped.iloc[:,0:12];
target   = combined_data_dropped.iloc[:,12:13];

features
target

train_features, test_features, train_target, test_target = train_test_split(features, target, 
                                                                test_size=0.33, random_state=123)

myLR = LinearRegression().fit(train_features, train_target);
myLR.score(train_features,train_target)
myLR.score(test_features,test_target)

myLR.coef_
myLR.intercept_
sns.lineplot(x=test_target.index, y=myLR.predict(test_features)[:,0])

test_target.plot()


ax = data_forum_electricity_resampled_interpolated_diff['2014-4-30'].reset_index().plot(y='bin_kwh')
data_forum_electricity_resampled_interpolated_diff['2014-12-30'].reset_index().plot( y='bin_kwh', ax=ax)